In [33]:
import numpy as np
import pandas as pd
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler

from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.selection import DropFeatures

from catboost import CatBoostRegressor

# MODELE JP BOX OFFICE

## Importation du pickle

In [34]:
data = pd.read_pickle("datasets/dataset_sql_cleaned.pkl")
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4546 entries, 0 to 4546
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   titre           4546 non-null   object  
 1   acteurs         4546 non-null   object  
 2   budget          4546 non-null   int64   
 3   compositeur     4546 non-null   object  
 4   semaine_fr      4546 non-null   object  
 5   semaine_usa     4546 non-null   object  
 6   duree           4546 non-null   int64   
 7   entrees_fr      4546 non-null   int64   
 8   franchise       4546 non-null   category
 9   genres          4546 non-null   category
 10  pegi_fr         4546 non-null   category
 11  pegi_usa        4546 non-null   category
 12  producteur      4546 non-null   object  
 13  realisateur     4546 non-null   object  
 14  entrees_usa     4546 non-null   int64   
 15  salles_fr       4546 non-null   int64   
 16  studio          4546 non-null   object  
 17  is_compositeur  454

## Netoyage db post netoyage

In [35]:
cols_drop = ["titre"]

data = data.drop(cols_drop, axis=1)

display(data.info())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
Index: 4546 entries, 0 to 4546
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   acteurs         4546 non-null   object  
 1   budget          4546 non-null   int64   
 2   compositeur     4546 non-null   object  
 3   semaine_fr      4546 non-null   object  
 4   semaine_usa     4546 non-null   object  
 5   duree           4546 non-null   int64   
 6   entrees_fr      4546 non-null   int64   
 7   franchise       4546 non-null   category
 8   genres          4546 non-null   category
 9   pegi_fr         4546 non-null   category
 10  pegi_usa        4546 non-null   category
 11  producteur      4546 non-null   object  
 12  realisateur     4546 non-null   object  
 13  entrees_usa     4546 non-null   int64   
 14  salles_fr       4546 non-null   int64   
 15  studio          4546 non-null   object  
 16  is_compositeur  4546 non-null   category
 17  annee           454

None

,acteurs,budget,compositeur,semaine_fr,semaine_usa,duree,entrees_fr,franchise,genres,pegi_fr,pegi_usa,producteur,realisateur,entrees_usa,salles_fr,studio,is_compositeur,annee,origine
0,"vincent cassel, gérard lanvin, simon pegg, qu...",90000000,-1,2009-07-03,2009-07-01,87,2403734,1,animation,Tous publics,PG,-1,-1,41690382,783,20th century fox,0,2009,etats-unis
1,"gerard butler, jodie foster",37000000,-1,2008-04-09,2008-04-04,95,243392,0,film familial,Tous publics,PG,-1,-1,13210579,399,snd,0,2008,etats-unis
2,"robert pattinson, kristen stewart",37000000,-1,2009-01-07,2008-11-21,120,755835,1,fantasy,Tous publics,PG,-1,-1,69637740,455,snd,0,2009,etats-unis
3,alain chabat,11378260,-1,2020-02-05,-1,98,112363,0,comédie,Tous publics,-1,-1,eric lartigau,-1,387,gaumont,0,2020,france
4,"scarlett johansson, demi moore",20000000,-1,2017-08-02,2017-06-16,101,66666,0,comédie,Tous publics,R,-1,-1,8004283,145,sony pictures,0,2017,etats-unis


### Métriques

In [36]:
# pred_test = pipe_cb.predict(X_test)

# score = pipe_cb.score(X_test, y_test)
# r2 = r2_score(y_test, pred_test)
# mse = mean_squared_error(y_test, pred_test)
# rmse = np.sqrt(mse)
# mae = mean_absolute_error(y_test, pred_test)

# print("Score :", score)
# print("Score R2 :", r2)
# print("Score MSE :", mse)
# print("Score RMSE", rmse)
# print("Score MAE :", mae)

#### Features importance

In [37]:
# preprocessed_features = pipe_cb.named_steps['columntransformer'].get_feature_names_out()
# cb_model = pipe_cb.named_steps['catboostregressor']

# feat_imp = pd.DataFrame({'features': preprocessed_features, 'score': cb_model.feature_importances_})
# feat_imp.sort_values(by=['score'], ascending=False, inplace=True)
# display(feat_imp)

## Pipeline modèle Lasso

In [38]:
# X = data.drop("entrees_fr", axis=1)
# y = data.entrees_premiere_semaine

# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42)

# object_cols = list(X.select_dtypes(include=["object"]).columns)
# display(object_cols)
# cat_cols = list(X.select_dtypes(include=["category"]).columns.drop(["annee"]))
# num_cols = list(X.select_dtypes(include=["int64"]).columns)
# year_col = ["annee"]

# # Ordinal encoding for ApprovalFY
# unique_years = sorted(data["annee"].unique())

# preprocessing = ColumnTransformer([
#         ("onehot", OneHotEncoder(), cat_cols),
#         # ("frequency", CountFrequencyEncoder(encoding_method="frequency", missing_values="ignore"), object_cols),
#         ("scaler", StandardScaler(), num_cols),
#         ("ordinal", OrdinalEncoder(categories=[unique_years], handle_unknown="use_encoded_value", unknown_value=2000), year_col),
#     ],
#     remainder="passthrough",
#     verbose_feature_names_out=False,
# )

# pre_fit = preprocessing.fit(X_train)
# fit_cols = pre_fit.get_feature_names_out()

# cat_indices = []
# for i, col_name in enumerate(fit_cols):
#     if col_name in cat_cols:
#         cat_indices.append(i)



# lasso_model = Lasso(alpha=1, random_state=42)

# pipe_lasso = make_pipeline(preprocessing, lasso_model)

# pipe_lasso.fit(X_train, y_train)

# # pipe_lasso.score(X_test, y_test)

## Pipeline modèle Linear Regression

In [39]:








# lr_model = LinearRegression()

# pipe_lr = make_pipeline(preprocessing, lr_model)

# display(pipe_lr)

# pipe_lr.fit(X_train, y_train)

# # test_score = pipe_lr.score(X_test, y_test)

## Pipeline modèle XGBoost

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from category_encoders.count import CountEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb



X = data.drop("entrees_fr", axis=1)
y = data.entrees_fr

# cols_drop = ["acteurs", "compositeur", "date", "pays", "producteur", "realisateur", "titre"]
# Liste des colonnes à garder
cols_keep = ["budget", "duree", "franchise", "genres", "pegi_fr", "pegi_usa", "entrees_usa", "salles_fr", "studio", "is_compositeur", "annee", "origine"]
X = X[cols_keep]

X["franchise"] = X["franchise"].astype(str)
X["genres"] = X["genres"].astype(str)
X["pegi_fr"] = X["pegi_fr"].astype(str)
X["pegi_usa"] = X["pegi_usa"].astype(str)
X["is_compositeur"] = X["is_compositeur"].astype(str)
# X["annee"] = X["annee"].astype(str)
X["origine"] = X["origine"].astype(str)


X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42)

object_cols = list(X.select_dtypes(include=["object"]).columns)
cat_cols = list(X.select_dtypes(include=["category"]).columns)#.drop(["annee"]))
num_cols = list(X.select_dtypes(include=["int64"]).columns.drop(["annee"]))
year_col = ["annee"]

# Ordinal encoding for ApprovalFY
unique_years = sorted(data["annee"].unique())
preprocessing = ColumnTransformer([
        ("onehot", OneHotEncoder(), cat_cols),
        ("frequency", CountEncoder(), object_cols),
        ("scaler", StandardScaler(), num_cols),
        ("ordinal", OrdinalEncoder(categories=[unique_years], handle_unknown="use_encoded_value", unknown_value=2000), year_col),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)

xgb_model = xgb.XGBRegressor(random_state=42)

# Définition de la grille des hyperparamètres à tester
# {'xgbregressor__colsample_bytree': 0.9, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.1, 'xgbregressor__max_depth': 4, 
#'xgbregressor__min_child_weight': 0.5, 'xgbregressor__n_estimators': 100, 'xgbregressor__reg_alpha': 0.05, 'xgbregressor__reg_lambda': 0.01,
# 'xgbregressor__scale_pos_weight': 1, 'xgbregressor__subsample': 1.0}

param_grid = {
    'xgbregressor__n_estimators': [100],#[50, 100, 150],
    'xgbregressor__learning_rate': [0.1],#[0.5, 0.1, 1.5],
    'xgbregressor__max_depth': [4],#[3.4, 4, 4.5],
    'xgbregressor__min_child_weight': [0.5],#[0.5, 1, 2],
    'xgbregressor__gamma': [0],#[0.0, 0.01, 0.02],
    'xgbregressor__subsample': [1.0],#[0.8, 0.9, 1.0],
    'xgbregressor__colsample_bytree': [0.9],#[0.8, 0.9, 1.0],
    'xgbregressor__reg_alpha': [0.05],#[0, 0.001, 0.005, 0.01, 0.05],
    'xgbregressor__reg_lambda': [0.01],#[0, 0.001, 0.005, 0.01, 0.05],
    'xgbregressor__scale_pos_weight': [0],#[1, 2, 3, 4, 5],
    # Ajoutez d'autres hyperparamètres à tester selon vos besoins
}

# Création du pipeline
pipe_xgb = make_pipeline(preprocessing, xgb_model)

# Recherche par grille des meilleurs hyperparamètres
grid_search = GridSearchCV(pipe_xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

# Prédiction sur l'ensemble de test avec les meilleurs paramètres
pred_test = grid_search.predict(X_test)

# Calcul des métriques d'évaluation
score = grid_search.score(X_test, y_test)
r2 = r2_score(y_test, pred_test)
mse = mean_squared_error(y_test, pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, pred_test)

print("Score :", score)
print("Score R2 :", r2)
print("Score MSE :", mse)
print("Score RMSE", rmse)
print("Score MAE :", mae)

Meilleurs hyperparamètres trouvés :
{'xgbregressor__colsample_bytree': 0.9, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.1, 'xgbregressor__max_depth': 4, 'xgbregressor__min_child_weight': 0.5, 'xgbregressor__n_estimators': 100, 'xgbregressor__reg_alpha': 0.05, 'xgbregressor__reg_lambda': 0.01, 'xgbregressor__scale_pos_weight': 0, 'xgbregressor__subsample': 1.0}
Score : -40514477251.11582
Score R2 : 0.7846639711988117
Score MSE : 40514477251.11582
Score RMSE 201282.08378073748
Score MAE : 105518.0915219485


In [41]:
# # Extraction des noms des fonctionnalités prétraitées
# preprocessed_features = grid_search.best_estimator_.named_steps['columntransformer'].get_feature_names_out()

# # Extraction du modèle XGBoost du meilleur estimateur
# xgb_model = grid_search.best_estimator_.named_steps['xgbregressor']

# # Création d'un DataFrame pour stocker l'importance des fonctionnalités
# feat_imp = pd.DataFrame({'features': preprocessed_features, 'score': xgb_model.feature_importances_})

# # Tri des fonctionnalités par importance
# feat_imp.sort_values(by=['score'], ascending=False, inplace=True)

# # Affichage du DataFrame contenant l'importance des fonctionnalités
# display(feat_imp)

## Pipeline modele Catboost

In [42]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, FunctionTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
# from category_encoders import CountFrequencyEncoder

# Vos données X et y
X = data.drop("entrees_fr", axis=1)
y = data.entrees_fr
cols_keep = ["budget", "duree", "franchise", "genres", "pegi_fr", "pegi_usa", "entrees_usa", "salles_fr", "studio", "is_compositeur", "annee", "origine"]
# cols_keep = ["budget", "duree", "franchise", "genres", "pegi_fr",  "salles_fr", "annee", "origine"]
X = X[cols_keep]

X["franchise"] = X["franchise"].astype(str)
X["genres"] = X["genres"].astype(str)
X["pegi_fr"] = X["pegi_fr"].astype(str)
X["origine"] = X["origine"].astype(str)

X["pegi_usa"] = X["pegi_usa"].astype(str)
X["is_compositeur"] = X["is_compositeur"].astype(str)
X["annee"] = X["annee"].astype(str)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42)

# Définition des colonnes catégorielles, numériques et année
object_cols = list(X.select_dtypes(include=["object"]).columns)
cat_cols = list(X.select_dtypes(include=["category"]).columns)#.drop(["annee"]))
num_cols = list(X.select_dtypes(include=["int64"]).columns.drop(["annee"]))
year_col = ["annee"]

# Définition du prétraitement des données
unique_years = sorted(data["annee"].unique())
preprocessing = ColumnTransformer([
        # ("onehot", OneHotEncoder(), cat_cols),
        ("frequency", CountFrequencyEncoder(encoding_method="frequency", missing_values="ignore"), object_cols),
        ("scaler", StandardScaler(), num_cols),
        ("ordinal", OrdinalEncoder(categories=[unique_years], handle_unknown="use_encoded_value", unknown_value=2000), year_col),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)

pre_fit = preprocessing.fit(X_train)
fit_cols = pre_fit.get_feature_names_out()

cat_indices = []
for i, col_name in enumerate(fit_cols):
    if col_name in cat_cols:
        cat_indices.append(i)

# Création du pipeline
pipe_cb = make_pipeline(preprocessing, CatBoostRegressor(one_hot_max_size=70, verbose=0, cat_features=cat_indices, random_state=42))

# Définition de la grille d'hyperparamètres à rechercher
# {'catboostregressor__depth': 7, 'catboostregressor__l2_leaf_reg': 0.6, 'catboostregressor__learning_rate': 0.055, 
#  'catboostregressor__n_estimators': 154, 'catboostregressor__random_strength': 0.31}

param_grid = {
    "catboostregressor__learning_rate": [0.05, 0.055],
    "catboostregressor__depth": [6.5, 7],
    "catboostregressor__l2_leaf_reg": [ 0.5, 0.6],
    "catboostregressor__n_estimators": [ 149, 154],
    "catboostregressor__random_strength": [ 0.30, 0.31],
}

# Recherche des meilleurs hyperparamètres avec validation croisée
grid_search = GridSearchCV(pipe_cb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Affichage des meilleurs hyperparamètres et de la performance
print("Meilleurs hyperparamètres trouvés:")
print(grid_search.best_params_)
print("Meilleur score sur les données d'entraînement:", grid_search.best_score_)

# Prédiction sur les données de test avec les meilleurs paramètres
pred_test = grid_search.best_estimator_.predict(X_test)

# Calcul des métriques de performance
score = grid_search.best_estimator_.score(X_test, y_test)
r2 = r2_score(y_test, pred_test)
mse = mean_squared_error(y_test, pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, pred_test)

print("Score :", score)
print("Score R2 :", r2)
print("Score RMSE", rmse)
print("Score MAE :", mae)


/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) pegi_fr.
  warnings.warn(
/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) pegi_fr.
  warnings.warn(
/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) pegi_fr.
  warnings.warn(
/Users/cyriljulliard/simplon/Movie_Popularity_Prediction/env/lib/python3.10/site-packages/feature_engine/encoding/base_encoder.py:260: UserWarning: During the encoding, NaN values were introduced in the feature(s) pegi_fr.
  warnings.warn(
/Users/cyriljulliard/simplon/Movie_Popul

Meilleurs hyperparamètres trouvés:
{'catboostregressor__depth': 7, 'catboostregressor__l2_leaf_reg': 0.5, 'catboostregressor__learning_rate': 0.055, 'catboostregressor__n_estimators': 154, 'catboostregressor__random_strength': 0.3}
Meilleur score sur les données d'entraînement: -45990342676.86013
Score : 0.7373722692738811
Score R2 : 0.7373722692738811
Score RMSE 222288.53345123585
Score MAE : 112614.09939791913


In [43]:
pipe_cb.fit(X_train, y_train)
preprocessed_features = pipe_cb.named_steps['columntransformer'].get_feature_names_out()
cb_model = pipe_cb.named_steps['catboostregressor']

feat_imp = pd.DataFrame({'features': preprocessed_features, 'score': cb_model.feature_importances_})
feat_imp.sort_values(by=['score'], ascending=False, inplace=True)
display(feat_imp)

,features,score
6,salles_fr,49.770122
7,annee,14.683647
4,budget,12.320000
5,duree,7.391976
1,genres,6.846947
0,franchise,6.543437
3,origine,1.824327
2,pegi_fr,0.619545


## Pipeline XGBoost par pays: Etats-Unis, France, other

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from category_encoders.count import CountEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb

X = data.drop("entrees_fr", axis=1)
y = data.entrees_fr

# cols_drop = ["acteurs", "compositeur", "date", "pays", "producteur", "realisateur", "titre"]
# Liste des colonnes à garder
cols_keep = ["budget", "duree", "franchise", "genres", "pegi_fr", "pegi_usa", "entrees_usa", "salles_fr", "studio", "is_compositeur", "annee", "origine"]
X = X[cols_keep]

X["franchise"] = X["franchise"].astype(str)
X["genres"] = X["genres"].astype(str)
X["pegi_fr"] = X["pegi_fr"].astype(str)
X["pegi_usa"] = X["pegi_usa"].astype(str)
X["is_compositeur"] = X["is_compositeur"].astype(str)
# X["annee"] = X["annee"].astype(str)
X["origine"] = X["origine"].astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42)

object_cols = list(X.select_dtypes(include=["object"]).columns)
cat_cols = list(X.select_dtypes(include=["category"]).columns)
num_cols = list(X.select_dtypes(include=["int64"]).columns.drop(["annee"]))
year_col = ["annee"]

# Ordinal encoding for ApprovalFY
unique_years = sorted(data["annee"].unique())
preprocessing = ColumnTransformer([
        ("onehot", OneHotEncoder(), cat_cols),
        ("frequency", CountEncoder(), object_cols),
        ("scaler", StandardScaler(), num_cols),
        ("ordinal", OrdinalEncoder(categories=[unique_years], handle_unknown="use_encoded_value", unknown_value=2000), year_col),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)

xgb_model = xgb.XGBRegressor(random_state=42)

# Définition de la grille des hyperparamètres à tester
# {'xgbregressor__colsample_bytree': 0.9, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.1, 'xgbregressor__max_depth': 4, 
#'xgbregressor__min_child_weight': 0.5, 'xgbregressor__n_estimators': 100, 'xgbregressor__reg_alpha': 0.05, 'xgbregressor__reg_lambda': 0.01,
# 'xgbregressor__scale_pos_weight': 1, 'xgbregressor__subsample': 1.0}

param_grid = {
    'xgbregressor__n_estimators': [100],#[50, 100, 150],
    'xgbregressor__learning_rate': [0.1],#[0.5, 0.1, 1.5],
    'xgbregressor__max_depth': [4],#[3.4, 4, 4.5],
    'xgbregressor__min_child_weight': [0.5],#[0.5, 1, 2],
    'xgbregressor__gamma': [0],#[0.0, 0.01, 0.02],
    'xgbregressor__subsample': [1.0],#[0.8, 0.9, 1.0],
    'xgbregressor__colsample_bytree': [0.9],#[0.8, 0.9, 1.0],
    'xgbregressor__reg_alpha': [0.05],#[0, 0.001, 0.005, 0.01, 0.05],
    'xgbregressor__reg_lambda': [0.01],#[0, 0.001, 0.005, 0.01, 0.05],
    'xgbregressor__scale_pos_weight': [0],#[1, 2, 3, 4, 5],

    # Ajoutez d'autres hyperparamètres à tester selon vos besoins
}

# Création du pipeline
pipe_xgb = make_pipeline(preprocessing, xgb_model)

# Séparation des données par origine
X_usa = X_train[X_train['origine'] == 'etats-unis']
y_usa = y_train[X_train['origine'] == 'etats-unis']
X_france = X_train[X_train['origine'] == 'france']
y_france = y_train[X_train['origine'] == 'france']
X_other = X_train[(X_train['origine'] != 'etats-unis') & (X_train['origine'] != 'france')]
y_other = y_train[(X_train['origine'] != 'etats-unis') & (X_train['origine'] != 'france')]

# Entraînement du modèle pour les films originaires des États-Unis
grid_search_usa = GridSearchCV(pipe_xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_usa.fit(X_usa, y_usa)
print("Meilleurs hyperparamètres pour les films originaires des États-Unis:")
print(grid_search_usa.best_params_)

# Entraînement du modèle pour les films originaires de France
grid_search_france = GridSearchCV(pipe_xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_france.fit(X_france, y_france)
print("Meilleurs hyperparamètres pour les films originaires de France:")
print(grid_search_france.best_params_)

# Entraînement du modèle pour les films originaires d'autres pays
grid_search_other = GridSearchCV(pipe_xgb, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_other.fit(X_other, y_other)
print("Meilleurs hyperparamètres pour les films originaires d'autres pays:")
print(grid_search_other.best_params_)

# Prédictions pour les films originaires des États-Unis
pred_usa = grid_search_usa.predict(X_test[X_test['origine'] == 'etats-unis'])
y_test_usa = y_test[X_test['origine'] == 'etats-unis']

# Prédictions pour les films originaires de France
pred_france = grid_search_france.predict(X_test[X_test['origine'] == 'france'])
y_test_france = y_test[X_test['origine'] == 'france']

# Prédictions pour les films originaires d'autres pays
pred_other = grid_search_other.predict(X_test[(X_test['origine'] != 'etats-unis') & (X_test['origine'] != 'france')])
y_test_other = y_test[(X_test['origine'] != 'etats-unis') & (X_test['origine'] != 'france')]

# Calcul des métriques pour chaque groupe
mse_usa = mean_squared_error(y_test_usa, pred_usa)
r2_usa = r2_score(y_test_usa, pred_usa)
print("Score pour les films originaires des États-Unis :", r2_usa)
print("Score RMSE pour les films originaires des États-Unis :", np.sqrt(mse_usa))

mse_france = mean_squared_error(y_test_france, pred_france)
r2_fr = r2_score(y_test_france, pred_france)
print("Score pour les films originaires de France :", r2_fr)
print("Score RMSE pour les films originaires de France :", np.sqrt(mse_france))

mse_other = mean_squared_error(y_test_other, pred_other)
r2_other = r2_score(y_test_other, pred_other)
print("Score pour les films originaires d'autres pays :", r2_other)
print("Score RMSE pour les films originaires d'autres pays :", np.sqrt(mse_other))


Meilleurs hyperparamètres pour les films originaires des États-Unis:
{'xgbregressor__colsample_bytree': 0.9, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.1, 'xgbregressor__max_depth': 4, 'xgbregressor__min_child_weight': 0.5, 'xgbregressor__n_estimators': 100, 'xgbregressor__reg_alpha': 0.05, 'xgbregressor__reg_lambda': 0.01, 'xgbregressor__scale_pos_weight': 0, 'xgbregressor__subsample': 1.0}
Meilleurs hyperparamètres pour les films originaires de France:
{'xgbregressor__colsample_bytree': 0.9, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.1, 'xgbregressor__max_depth': 4, 'xgbregressor__min_child_weight': 0.5, 'xgbregressor__n_estimators': 100, 'xgbregressor__reg_alpha': 0.05, 'xgbregressor__reg_lambda': 0.01, 'xgbregressor__scale_pos_weight': 0, 'xgbregressor__subsample': 1.0}
Meilleurs hyperparamètres pour les films originaires d'autres pays:
{'xgbregressor__colsample_bytree': 0.9, 'xgbregressor__gamma': 0, 'xgbregressor__learning_rate': 0.1, 'xgbregressor

## Export du modele Catboost

In [45]:
with open("models/model_cb.pkl", "wb") as f:
  pickle.dump(pipe_cb, f)
f.close()